In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as pdr
import math
import csv

# Import EDD utils to import study as pandas dataframe
from edd_utils import login, export_study

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from datetime import datetime
import seaborn as sns; sns.set(color_codes=True)
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label, Layout ,Button,AppLayout, jslink, IntText, IntSlider
import matplotlib.colors as mcolors
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly
%matplotlib inline

In [1]:
FILE='DBTL0/DBTL0-1.csv'
df
df1=df_renamed.copy()

Line_Name = {}
data = []
strains = df1['Line Name'].unique()
size=len(strains)

fig, axs = plt.subplots(int(size/12), 12, figsize=(21, 14),sharey=True, sharex=True) 

nrow = math.ceil(size/4)
ncol = 4

for strain in strains:
    Line_Name[strain]=df1[df1['Line Name']==strain]
    
for k, strain in enumerate(strains,start=1):
    nlog=np.log(Line_Name[strain]['Value'])
    time=Line_Name[strain]['Time']

    df1 = pd.concat([time, nlog], axis=1)
    time_range2=np.arange(0,len(df1['Time']))*5
    
    slopes=[]
    r2s=[]
    pvals=[]
    intercepts=[]
    rolling_window = 60
    
    # Perform rolling linear regression
    for i in range(len(df1) - rolling_window + 1):
        subset = df1.iloc[i:i+rolling_window]
        X = sm.add_constant(subset['Time'])  # Add a constant term for the intercept
        y = subset['Value']
        model = sm.OLS(y, X).fit()
        
        slope = model.params['Time']  # Get the slope coefficient
        slopes.append(slope)
        intercept = model.params['const'] # Get the intercept
        intercepts.append(intercept)
        r2=model.rsquared
        r2s.append(r2)
        pval=model.pvalues
        pvals.append(pval)

    max1=max(slopes)
    umax_pos=(slopes.index(max(slopes)))
    pval=pvals[umax_pos]
    r2=r2s[umax_pos]
    intercept=intercepts[umax_pos]
    y_pred = max1*time+intercept
    
    result=(strain, max1, r2)
    print(result)

    data.append(result)
    
    ax=axs.flatten()[k-1]
    ax.set_title(strain, size=10)
    ax.plot(time_range2, nlog)
    ax.plot(time_range2, y_pred)
    ax.set_ylim([0, 4])
    plt.tight_layout()

NameError: name 'df_renamed' is not defined